## Work Flow For Prediction:
1. Read the sample images
2. extract the face out of it
3. feed it to extract features
4. find out the cosine similarity
5. get the highest maych feature image and name

In [1]:
from tensorflow.keras.preprocessing import image
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace
import numpy as np
from mtcnn import MTCNN #Face detection
from PIL import Image # image Preprocess
import cv2 #image read 
import pickle



In [2]:
features = pickle.load(open(r"C:\Users\Dell\OneDrive\Desktop\Resume\celeb\WhichCelebAreYou\features.pkl" , "rb"))
features = np.array(features)

In [3]:
filename = pickle.load(open(r"C:\Users\Dell\OneDrive\Desktop\Resume\celeb\WhichCelebAreYou\filename.pkl" , "rb"))

In [4]:
model = VGGFace(model = "resnet50" , include_top = False , input_shape=(224,224,3) , pooling = "avg")

In [5]:
detector = MTCNN() # for face detection

In [6]:
def feature_test(path,model):
  print(path)
  sample_img = cv2.imread(path)     #reading image in np.array
  #print(sample_img)
  results = detector.detect_faces(sample_img)  #MTCNN used to detect the coordiantes of the face
  x,y,width,height =  results[0]["box"]         # gives the position of face 
  face = sample_img[y:y+height , x:x+height]    # crop the pixel to the onlt to have face
  image = Image.fromarray(face)                 # convert pixel to img
  image = image.resize((224,224))               # resize to enter into CNN
  face_array = np.array(image)                  # getting back to array 
  face_array  = face_array.astype("float32")    # converting values to float
  expanded_img = np.expand_dims(face_array,axis = 0)    # having desired format (1,224,224,3)
  #print(expanded_img.shape)                        
  preprocess_img = preprocess_input(expanded_img)       
  result = model.predict(preprocess_img).flatten()      #Extracting Features

  return result

In [20]:
result = feature_test(r"C:\Users\Dell\OneDrive\Desktop\Resume\celeb\WhichCelebAreYou\Bollywood_celeb_face_localized\Sample\Capture.JPG",model)
result.shape


C:\Users\Dell\OneDrive\Desktop\Resume\celeb\WhichCelebAreYou\Bollywood_celeb_face_localized\Sample\Capture.JPG


(2048,)

## CALCULATING COSINE SIMILARITY 

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
similarity = []
for feature in features:
  similarity.append(list(cosine_similarity(result.reshape(1,-1) , feature.reshape(1,-1)))[0][0])
similarity_score = sorted(list(enumerate(similarity)) , reverse=True , key = lambda x: x[1])[0][0]



In [23]:
similarity_score

8483

In [44]:
import os
path = filename[similarity_score]
final_path = os.path.join(r"C:/Users/Dell/OneDrive/Desktop/Resume/celeb\WhichCelebAreYou" , path)
temp = cv2.imread(r"C:\Users\Dell\OneDrive\Desktop\Resume\celeb\WhichCelebAreYou\content\bollywood_celeb_faces_0\Anushka_Shetty\Anushka_Shetty.25.jpg")
cv2.imshow("img",temp)
cv2.waitKey(2000)



-1